In [11]:
def load_annotation(label_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()

    if not lines or not lines[0].strip():
        print(f"[INFO] File label kosong (tidak ada PPE): {label_path}")
        # bisa return bbox dummy dengan class 0 atau None tergantung use case
        return 0, [0.5, 0.5, 0.1, 0.1]

    try:
        parts = lines[0].strip().split()
        class_id = int(parts[0])
        bbox = list(map(float, parts[1:5]))
        return class_id, bbox
    except Exception as e:
        print(f"[ERROR] Gagal parsing label di {label_path}: {e}")
        return 0, [0.5, 0.5, 0.1, 0.1]


In [3]:
import tensorflow as tf
import cv2
import os
import numpy as np

class YoloDataset(tf.keras.utils.Sequence):
    def __init__(self, image_dir, label_dir, batch_size=16, img_size=224, num_classes=8):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.num_classes = num_classes
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
    
    def __len__(self):
        return len(self.image_files) // self.batch_size
    
    def __getitem__(self, idx):
        batch_files = self.image_files[idx*self.batch_size:(idx+1)*self.batch_size]
        images = []
        classes = []
        bboxes = []
        
        for file in batch_files:
            # Load and preprocess image
            img_path = os.path.join(self.image_dir, file)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (self.img_size, self.img_size))
            img = img / 255.0
            images.append(img)
            
            # Load annotation
            txt_file = file.replace('.jpg', '.txt')
            class_id, bbox = load_annotation(os.path.join(self.label_dir, txt_file))
            classes.append(class_id)
            bboxes.append(bbox)
        
        images = np.array(images, dtype=np.float32)
        classes = tf.keras.utils.to_categorical(classes, num_classes=self.num_classes)
        bboxes = np.array(bboxes, dtype=np.float32)
        
        return images, {'class_output': classes, 'bbox_output': bboxes}


In [4]:
from tensorflow.keras import layers, models

def create_model(input_shape=(224,224,3), num_classes=8):
    inputs = layers.Input(shape=input_shape)
    
    # Backbone CNN sederhana
    x = layers.Conv2D(32, (3,3), activation='relu')(inputs)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(64, (3,3), activation='relu')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Conv2D(128, (3,3), activation='relu')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(256, activation='relu')(x)
    
    # Output klasifikasi
    class_output = layers.Dense(num_classes, activation='softmax', name='class_output')(x)
    
    # Output bounding box (x_center, y_center, width, height)
    bbox_output = layers.Dense(4, activation='sigmoid', name='bbox_output')(x)
    
    model = models.Model(inputs=inputs, outputs=[class_output, bbox_output])
    return model


In [12]:
# Buat model
model = create_model()

# Compile dengan loss yang berbeda untuk 2 output
model.compile(
    optimizer='adam',
    loss={
        'class_output': 'categorical_crossentropy',
        'bbox_output': 'mse'
    },
    metrics={
        'class_output': 'accuracy',
        'bbox_output': 'mse'
    }
)

# Direktori data (sesuaikan)
train_image_dir = 'dataset/train/images'
train_label_dir = 'dataset/train/labels'
val_image_dir = 'dataset/valid/images'
val_label_dir = 'dataset/valid/labels'

# Buat dataset generator
train_gen = YoloDataset(train_image_dir, train_label_dir, batch_size=16, img_size=224, num_classes=8)
val_gen = YoloDataset(val_image_dir, val_label_dir, batch_size=16, img_size=224, num_classes=8)

# Training
model.fit(train_gen, validation_data=val_gen, epochs=20)


Epoch 1/20
63/78 ━━━━━━━━━━━━━━━━━━━━ 10s 697ms/step - bbox_output_loss: 0.0399 - bbox_output_mse: 0.0399 - class_output_accuracy: 0.5451 - class_output_loss: 2.1933 - loss: 2.2332[INFO] File label kosong (tidak ada PPE): dataset/train/labels/image_244_jpg.rf.f858a0066ef9b0dd3a830074e4d444f9.txt
78/78 ━━━━━━━━━━━━━━━━━━━━ 57s 718ms/step - bbox_output_loss: 0.0382 - bbox_output_mse: 0.0382 - class_output_accuracy: 0.5583 - class_output_loss: 2.0331 - loss: 2.0713 - val_bbox_output_loss: 0.0348 - val_bbox_output_mse: 0.0348 - val_class_output_accuracy: 0.6786 - val_class_output_loss: 0.8635 - val_loss: 0.8983
Epoch 2/20
31/78 ━━━━━━━━━━━━━━━━━━━━ 33s 715ms/step - bbox_output_loss: 0.0261 - bbox_output_mse: 0.0261 - class_output_accuracy: 0.6463 - class_output_loss: 0.9905 - loss: 1.0165[INFO] File label kosong (tidak ada PPE): dataset/train/labels/image_244_jpg.rf.f858a0066ef9b0dd3a830074e4d444f9.txt
78/78 ━━━━━━━━━━━━━━━━━━━━ 64s 819ms/step - bbox_output_loss: 0.0249 - bbox_output_mse: 

In [16]:
model.save("ppe_model.h5")

model.save("ppe_model.keras")  # ini format baru Keras 3


In [19]:
# Buat test generator
test_image_dir = 'dataset/test/images'
test_label_dir = 'dataset/test/labels'

test_gen = YoloDataset(test_image_dir, test_label_dir, batch_size=16, img_size=224, num_classes=8)

# Evaluasi model
loss, class_loss, bbox_loss, class_acc, bbox_mse = model.evaluate(test_gen, return_dict=False)

print("Total Loss:", loss)
print("Class Loss:", class_loss)
print("BBox Loss:", bbox_loss)
print("Class Accuracy:", class_acc)
print("BBox MSE:", bbox_mse)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 263ms/step - bbox_output_loss: 0.0323 - bbox_output_mse: 0.0323 - class_output_accuracy: 0.6432 - class_output_loss: 1.6373 - loss: 1.6695
Total Loss: 1.7123584747314453
Class Loss: 1.681443691253662
BBox Loss: 0.03091474436223507
Class Accuracy: 0.03091474436223507
BBox MSE: 0.6458333134651184
